# Yesterday's best deck bot
Build a reddit bot that posts yesterday's best deck

- [x] scrape appropriate data from balco-lor.com
- [ ] combine files
- [ ] clean data
- [ ] analyze data
- [ ] build post
- [ ] post to r/lor

## 1. scrape data

In [78]:
import time
import shutil
import pandas as pd
import glob
import datetime
import pandas as pd
import praw
import json

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
import os

os.getcwd()

'C:\\Users\\jagod\\github\\LoR-muTable\\python'

In [2]:
#set up selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.balco-lor.com/ranked/')

try:
    wait = WebDriverWait(driver, 60) #define max wait time
    element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="filter_dates"]/div/span/span[6]'))) #wait until scroller appears
finally:
    left = driver.find_element(By.XPATH, '//*[@id="filter_dates"]/div/span/span[6]') #define left scroller
    element = driver.find_element(By.XPATH, '//*[@id="filter_dates"]/div/span/span[7]') # define right element
    time.sleep(1)
    webdriver.ActionChains(driver).click_and_hold(left).pause(0).move_to_element_with_offset(element, -20, 0).pause(0).release().perform() # move left scroller to where it needs to be relative to right scroller
    
# download region files for yesterday (approx)
regions = ['europe', 'americas', 'asia']
for i in regions:
    button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/div[1]/div/div/div[1]') # open region selection box
    button.click()
    time.sleep(1)
    region = driver.find_element(By.XPATH, f'//div[@data-value="{i}"]') # click appropriate region
    region.click()
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="get_info"]').click() #click to load the data
    time.sleep(60) #make it so that when XPATH "//*[@id="DataTables_Table_2"]/tbody/tr[1]/td[6]" matches "i", you hit the download button
    driver.find_element(By.XPATH, '//*[@id="dl_csv"]').click() # click to download
    time.sleep(5)

### 2. combine files

In [44]:
# get relevant dates
today = datetime.date.today()
yesterday = today-datetime.timedelta(days=1)

yesterday_dash = yesterday.strftime('%Y-%m-%d')

In [12]:
# dates = f'{yesterday}-{today}'

In [48]:
def combine_csv(dates):
    path = r"../../../downloads/" # use your path
    all_files = glob.glob(path + "/*lor_ranked_data*.csv")

    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [49]:
df = combine_csv(dates)

### 3. clean data

In [67]:
df['game_start_time_utc'] = pd.to_datetime(df['game_start_time_utc']) # convert to datetime
df['game_start_time_utc'] = df['game_start_time_utc'].dt.strftime('%Y-%m-%d') # format it how you want
mask = (df['game_start_time_utc'] == yesterday_dash) # create mask
df = df.loc[mask] # filter on mask

### 4. analyze data

In [68]:
archetypes = df['deck_code'].value_counts()
archetypes = archetypes[archetypes > 300]
popular_list = list(archetypes.index)
x = df.loc[df['deck_code'].isin(popular_list)]

In [69]:
df_champ = x[x['game_outcome'] != "tie"] # get rid of ties
df_champ = df_champ.groupby(['deck_code', 'game_outcome']).size() # group deck codes by outcome
df_champ = df_champ.groupby(level=0).transform(lambda x: (x / x.sum()).round(3)*100) # get winrate
df_champ.drop('loss', level=1, axis=0, inplace=True) # drop losses
df_champ = df_champ.droplevel(level=1) # not sure
df_champ = df_champ.sort_values(ascending=False) # sort so good stuff at top
df_champ = pd.DataFrame({'deck_code':df_champ.index, 'winrate':df_champ.values}) #dataframe w/ column titles

In [70]:
df_ = df[['deck_code', 'archetype']].copy() # make a dataframe with just the stuff you need
_ = pd.merge(df_champ, df_, on=['deck_code'], copy=False).drop_duplicates().reset_index(drop=True)[:20] # get the archetypes for top decks, drop dupes, reset index, keep top 10

In [71]:
# get sample size for each deck
z = df.value_counts('deck_code')
z = pd.DataFrame(z, columns = ['n_games'])
_ = _.merge(z, how='inner', on= 'deck_code')

In [74]:
# define a bunch of variables
top_archetype = _['archetype'][0]
top_archetype_plays = df['archetype'].value_counts()[top_archetype]
top_winrate = round(_['winrate'][0])
top_deck_n_games = _['n_games'][0]
pdf_n_games_total = df.shape[0]

deck_wr0 = round(_['winrate'][0])
deck_wr1 = round(_['winrate'][1])
deck_wr2 = round(_['winrate'][2])
# deck_wr3 = round(_['winrate'][3])
# deck_wr4 = round(_['winrate'][4])
# deck_wr5 = round(_['winrate'][5])
# deck_wr6 = round(_['winrate'][6])

deck_code0 = _['deck_code'][0]
deck_code1 = _['deck_code'][1]
deck_code2 = _['deck_code'][2]
# deck_code3 = _['deck_code'][3]
# deck_code4 = _['deck_code'][4]
# deck_code5 = _['deck_code'][5]
# deck_code6 = _['deck_code'][6]

deck_archetype0 = _['archetype'][0]
deck_archetype1 = _['archetype'][1]
deck_archetype2 = _['archetype'][2]
# deck_archetype3 = _['archetype'][3]
# deck_archetype4 = _['archetype'][4]
# deck_archetype5 = _['archetype'][5]
# deck_archetype6 = _['archetype'][6]

### generate post

In [75]:
print(f'Yesterday\'s top deck is {top_archetype}, with a {top_winrate}% winrate. \
      \nOut of {pdf_n_games_total} matches analyzed, {top_archetype} was played {top_archetype_plays} times ({round((top_archetype_plays/pdf_n_games_total)*100)}%). Its top deck was played {top_deck_n_games} times with an overall {top_winrate}% winrate. \
      \n\n|Winrate|Archetype|Deck Code|\
      \n|:-|:-|:-| \
      \n|{deck_wr0}%|{deck_archetype0}|{deck_code0}| \
      \n|{deck_wr1}%|{deck_archetype1}|{deck_code1}|  \
      \n|{deck_wr2}%|{deck_archetype2}|{deck_code2}|\
      \n\nHappy climbing, and all hail [Balco](https://lor-meta.com/) for the data!')

Yesterday's top deck is Miss Fortune Twisted Fate (NX), with a 64% winrate.       
Out of 44938 matches analyzed, Miss Fortune Twisted Fate (NX) was played 4781 times (11%). Its top deck was played 342 times with an overall 64% winrate.       

|Winrate|Archetype|Deck Code|      
|:-|:-|:-|       
|64%|Miss Fortune Twisted Fate (NX)|CICQCAQDAMAQKBQBAIDAMFI6AQAQGAQMB4UAIAQGCYNCMOQCAEAQGJIBAIDC2AA|       
|59%|Miss Fortune Twisted Fate (NX)|CIDACAQDAMAQKBQBAEDAMFIBAYBROBABAMBAYDZIAQBAMFQ2EY5AEAICAYWQCBQGDYAA|        
|56%|Miss Fortune Twisted Fate (NX)|CICQCAQDAMAQKBQBAEDAMFIEAIDBMGRGHICQCAYCBQHSKKACAEBAMLIBAYDB4AA|      

Happy climbing, and all hail [Balco](https://lor-meta.com/) for the data!


### post to reddit

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

x = get_keys('praw.env')

In [ ]:
reddit = praw.Reddit(client_id = x['client_id'],
                     client_secret = x['client_secret'],
                     user_agent = x['user_agent'],
                     username = x['username'],
                     password = x['password'])

In [87]:
x = get_keys('praw.env')

In [93]:
x['client_id']

'Lw3fTHOcCYwmajAf-b5HiQ'

In [92]:
x.keys()

dict_keys(['client_id', 'client_secret', 'user_agent', 'username', 'password'])